# Form the NB Regression Random Intercept Model: Simulated set

Next we may try joint estimation using Newton's + IPOPT. 

    I create new structures `NBCopulaVCObs`, `NBCopulaVCModel`
    



In [31]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.5

r = 1
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Vector{Vector{Float64}}:
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [2.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 3.0, 1.0]
 [0.0, 0.0, 1.0, 0.0, 2.0]
 [3.0, 3.0, 2.0, 1.0, 1.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [3.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 2.0]
 [0.0, 1.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 1.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 ⋮
 [0.0, 0.0, 1.0, 0.0, 1.0]
 [1.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 1.0, 0.0, 0.0, 1.0]
 [0.0, 0.0, 1.0, 2.0, 0.0]
 [1.0, 0.0, 0.0, 0.0, 3.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 2.0, 1.0]
 [2.0, 2.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 0.0, 0.0]
 [0.0, 0.0, 0.0, 2.0, 0.0]
 [2.0, 0.0, 1.0, 0.0, 0.0]
 [0.0, 0.0, 1.0, 0.0, 0.0]

In [32]:
d = NegativeBinomial()
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{NBCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = NBCopulaVCObs(y, X, V, d, link)
end
gcm = NBCopulaVCModel(gcs);
# gcm.r[1] = r

In [33]:
initialize_model!(gcm)
# @show gcm.β
# @show gcm.Σ
@show gcm.r

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -55442.78384099386 39999
2 -55442.78384099386 -55442.78384099386 39999
initializing variance components using MM-Algorithm
gcm.Σ = [1.8072767693765704e-5, 0.055284084750823415]
gcm.r = [1.0]


1-element Vector{Float64}:
 1.0

In [34]:
β_true

-0.8472978603872034

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [35]:
GLMCopula.loglikelihood!(gcm, true, true)

-55348.28950500059

## Try Quasi-Newton after specifying gradient of r

to-do: 

1. added field $r$ to model structure $gcm.r$, change access to r this way instead of through the distribution
2. add $gc.\nabla r$ in loglikelihood function


## Newton after specifying the hessian term too

3. add $gc.Hr$ in loglikelihood function

In [36]:
# Quasi-Newton
@time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "limited-memory"))

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

┌ Warning: Optimization unsuccesful; got UserLimit
└ @ GLMCopula /Users/biona001/.julia/dev/GLMCopula/src/parameter_estimation/fit_nb.jl:30


-56346.235198594295

In [37]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = -0.4524277782239204; true β = -0.8472978603872034
estimated variance component 1 = 6.641082089808847e-8; true variance component 1 = 0.1
estimated variance component 2 = 0.69294167002372; true variance component 2 = 0.5
estimated r = 1.1084050216812205; true r = 1


**If initializing VCs to 1**:
+ With true $r = 1$
    - good estimates of $\beta$, VC parameters, and r
    - sometimes does crazy line search. 
+ With true $r = 2$
    - VC parameters goes to 0, esimated $\beta$ is wrong, and estimated r is 1.17. 

**If initializing VCs by update_$\Sigma$**:
+ With true $r = 1$
    - one VC parameters becomes basically 0 after initialization
    - bad estimated beta (-0.45 vs -0.84 truth)
    - estimated VC = 0.0 and 0.69 (truth = 0.1 and 0.5)
    - estimated r = 1.1

## Newton

In [8]:
# @time GLMCopula.fit!(gcm, IpoptSolver(print_level = 5, max_iter = 100, hessian_approximation = "exact"))

In [9]:
println("estimated β = $(gcm.β[1]); true β = $β_true")
println("estimated variance component 1 = $(gcm.Σ[1]); true variance component 1 = $variance_component_1")
println("estimated variance component 2 = $(gcm.Σ[2]); true variance component 2 = $variance_component_2")
println("estimated r = $(gcm.r[1]); true r = $r");

estimated β = 635.6293856671999; true β = 1.4552872326068422
estimated variance component 1 = 4.995785928691524; true variance component 1 = 0.1
estimated variance component 2 = 0.014080169494087566; true variance component 2 = 0.5
estimated r = 1.1030329320877854; true r = 10
